In [ ]:
import os
import concurrent.futures
import subprocess
import platform

def 검색_작업(드라이브, 키워드):
    일치하는_파일들 = []
    for 루트, 디렉토리들, 파일들 in os.walk(f"{드라이브}:\\"):
        for 파일 in 파일들:
            if 키워드.lower() in 파일.lower():
                전체_경로 = os.path.join(루트, 파일)
                일치하는_파일들.append(전체_경로)
    return 일치하는_파일들

def 파일_검색(키워드, 검색_드라이브):
    일치하는_파일들 = []
    with concurrent.futures.ThreadPoolExecutor() as executor:
        결과 = list(executor.map(lambda 드라이브: 검색_작업(드라이브, 키워드), 검색_드라이브))
    for 파일_목록 in 결과:
        일치하는_파일들.extend(파일_목록)
    return 일치하는_파일들

def 파일_목록_표시(파일_목록):
    print("\n검색된 파일 목록:")
    print("------------------------------------------------")
    for 인덱스, 파일 in enumerate(파일_목록, 1):
        print(f"{인덱스}. {파일}")
    print("------------------------------------------------")
    if not 파일_목록:
        print("검색된 파일이 없습니다.")

def 파일_선택_및_실행(파일_목록):
    while True:
        선택 = int(input("\n실행할 파일의 번호를 입력하세요 (종료하려면 0을 입력): "))
        if 선택 == 0:
            print("프로그램을 종료합니다.")
            break
        elif 1 <= 선택 <= len(파일_목록):
            파일_실행(파일_목록[선택 - 1])
        else:
            print("번호가 유효하지 않습니다.")

def 파일_실행(파일_경로):
    if platform.system() == 'Windows':
        os.startfile(파일_경로)
    elif platform.system() == 'Linux' or platform.system() == 'Darwin':
        subprocess.run(['open' if platform.system() == 'Darwin' else 'xdg-open', 파일_경로])

def 메인():
    검색_드라이브 = ['C', 'D']  # C와 D 드라이브 검색
    키워드 = input("파일 검색을 위한 키워드를 입력하세요: ")
    찾은_파일들 = 파일_검색(키워드, 검색_드라이브)
    파일_목록_표시(찾은_파일들)
    if 찾은_파일들:
        파일_선택_및_실행(찾은_파일들)

# 스크립트 실행
메인()


파일 검색을 위한 키워드를 입력하세요:  압축


In [ ]:
# 동영상찾기

In [ ]:
import os
from datetime import datetime

# 동영상 및 사진 파일 확장자 리스트
default_video_extensions = ['.mp4', '.avi', '.mov', '.mkv']
default_photo_extensions = ['.jpg', '.jpeg', '.png', '.gif']

# 모든 파일 리스트를 저장할 리스트
all_files = []

# 드라이브 경로들
# drive_paths = ['C:\\', 'D:\\']
drive_paths = ['C:\\', 'D:\\']

# 디버그 모드 설정
DEBUG_MODE = False

def search_files(keyword=None, extensions=None):
    global all_files
    all_files = []

    if extensions is None:
        extensions = default_video_extensions + default_photo_extensions

    # 드라이브의 모든 파일 탐색
    for drive in drive_paths:
        for root, dirs, files in os.walk(drive):
            for file in files:
                if any(file.lower().endswith(ext) for ext in extensions):
                    if keyword is None or keyword.lower() in file.lower():
                        file_path = os.path.join(root, file)
                        try:
                            file_mtime = os.path.getmtime(file_path)
                            all_files.append((file_path, datetime.fromtimestamp(file_mtime)))
                        except (FileNotFoundError, PermissionError) as e:
                            if DEBUG_MODE:
                                print(f"파일 정보를 가져오는 중 오류 발생: {file_path} - {e}")

    # 파일을 생성 날짜별로 정렬
    all_files.sort(key=lambda x: x[1])

def display_files():
    for i, (file_path, file_mtime) in enumerate(all_files):
        print(f"{i + 1}. {file_path} (생성 날짜: {file_mtime})")

def open_file(index):
    if 0 <= index < len(all_files):
        file_path = all_files[index][0]
        print(f"{file_path} 실행 중...")
        try:
            os.startfile(file_path)
        except Exception as e:
            print(f"파일을 실행하는 중 오류 발생: {e}")
    else:
        print("잘못된 번호입니다. 다시 시도해 주세요.")

def main():
    while True:
        try:
            keyword = input("검색할 키워드를 입력하세요 (전체 검색하려면 엔터): ")
            ext_input = input("검색할 확장자를 입력하세요 (쉼표로 구분, 예: .mp4,.jpg 전체 검색하려면 엔터): ")
            extensions = [ext.strip() for ext_input in ext_input.split(',')] if ext_input else None

            print("파일을 검색 중입니다...")
            search_files(keyword, extensions)
            if not all_files:
                print("검색 결과가 없습니다.")
                continue

            display_files()

            while True:
                selection = input("실행할 파일 번호를 선택하세요 (종료하려면 'exit' 입력, 새로운 검색을 원하면 'new' 입력): ")
                if selection.lower() == 'exit':
                    return
                if selection.lower() == 'new':
                    break
                try:
                    index = int(selection) - 1
                    open_file(index)
                except ValueError:
                    print("잘못된 입력입니다. 숫자를 입력해 주세요.")
                except Exception as e:
                    print(f"오류 발생: {e}")

        except Exception as e:
            print(f"오류 발생: {e}")

if __name__ == "__main__":
    main()

